## TASK
Implement a system that:
1. will automatically download files from an FTP address
2. will process the data and get a ready CSV table
3. will send this table to the specified FTP address

### importing required libraries

In [ ]:
import ftplib
import os
import zipfile
import mysql.connector as connection
import pandas as pd
import pyunpack

### connect by ftp and download archive

In [134]:
ftp_host = "**************"
ftp_user = "**************"
ftp_pass = "**************"

ftp = ftplib.FTP(ftp_host, ftp_user, ftp_pass) #FTP connection
ftp.cwd("/")
savedir="C:/task"
os.chdir(savedir)

filename = "task.rar"

rar_file = open(filename, "wb")
ftp.retrbinary("RETR " + filename, rar_file.write) #save file in local directory
rar_file.close()

In [135]:
pyunpack.Archive("task.rar").extractall("C:/task") #unpack archive

### connect to MySql DB

In [136]:
config = {
  'user': 'root',
  'password': "**************",
  'host': '127.0.0.1',
  'database': 'manufacture',
  'raise_on_warnings': True
}

cnx = connection.connect(**config)

### main query

In [150]:
query = """
    SELECT product.main_part_number, product.manufacturer, product.category, product.origin, 
    CAST(REPLACE(quantity.quantity, '>', '') AS UNSIGNED) AS quantity, price.price,
    IFNULL(deposit.deposit,0) AS deposit, (price.price + IFNULL(deposit.deposit,0)) AS total_price
    FROM product
    LEFT JOIN price ON product.part_number = price.part_number 
    LEFT JOIN deposit ON product.part_number = deposit.part_number 
    LEFT JOIN quantity ON product.part_number = quantity.part_number 
    WHERE quantity.warehouse IN ('A', 'H', 'J', '3', '9') AND (quantity.quantity NOT IN ('0')) 
    AND ((price.price + IFNULL(deposit.deposit,0)) >= 2.00)
    ORDER BY quantity.warehouse
"""

In [151]:
result_dataFrame = pd.read_sql(query, cnx) #Show result in dataFrame
result_dataFrame.head()

,main_part_number,manufacturer,category,origin,quantity,price,deposit,total_price
0,WT89691,HART FILTRY I,Filtry paliwa,CN,1,114.84,0.0,114.84
1,2341801,TEXTAR,Klocki hamulcowe,RO,1,1678.06,0.0,1678.06
2,51391-S5A-024*,HART EL.MET-GUM.III,Tuleje,PL,2,107.08,0.0,107.08
3,771 0320 30,FAG,Przeguby,PL,1,999.15,0.0,999.15
4,1 987 946 062,BOSCH,Paski micro,PL,1,159.49,0.0,159.49


### save to .csv file and upload by ftp

In [152]:
result_dataFrame.to_csv('./ftp_rezults_VictoriaDyshlyuk.csv', sep ='\t', index=False, encoding='UTF-8') #Save .csv

In [153]:
ftp = ftplib.FTP(ftp_host, ftp_user, ftp_pass) #connect ftp

In [154]:
directory_list = ftp.nlst()
print(directory_list)

['task.rar', 'complete', '.', ' complete_result.csv', ' complete\\result.csv', '..']


In [156]:
result_file = "C:/task/ftp_rezults_VictoriaDyshlyuk.csv"

#ftp.cwd('/complete/')
#ftp.mkd('VictoriaDyshlyuk')
ftp.cwd('/complete/VictoriaDyshlyuk/')
ftp.storbinary('STOR ftp_rezults_VictoriaDyshlyuk.csv', open(result_file, 'rb')) #Save in ftp folder


'226 Transfer complete'